# RoboHive Environment Registration & Customization

## Register
Follow Gym registration API to register a RoboHive Environment

In [ ]:
import gym
from gym.envs.registration import register
import os

In [ ]:
# Hand Manipulation Suite: Open door
from robohive.envs.hands.door_v1 import DoorEnvV1
register(
    id='DemoDoor-v1',
    entry_point='robohive.envs.hands:DoorEnvV1',
    max_episode_steps=100,
    kwargs={
        'model_path':'../envs/hands/assets/DAPG_door.xml',
    }
)

Lets test the newly registered environment

In [ ]:
env = gym.make('DemoDoor-v1')
env.reset()
print(f"Time={env.time:.2f}", end=", ")
for _ in range(10):
    env.step(env.action_space.sample()) # take a random action
    print(f"{env.time:.2f}", end=", ")
env.close()

## Configure
RoboHive aggressively uses `kwargs` to configure environments. There are three different ways to configure environments in RoboHive 

### 1. Passing `kwargs` during registration
In the examples below `kwargs` are used to specify which `robot_site_name` to use for reaching to which `target_site_name`. Additionally the randomization range of the target is specified as `target_xyz_range`.

In [ ]:
# Franka arm to reach random targets
register(
    id='DemoFrankaReachRandom-v0',
    entry_point='robohive.envs.arms.reach_base_v0:ReachBaseV0',
    max_episode_steps=50, #50steps*40Skip*2ms = 4s
    kwargs={
        'model_path': '../envs/arms/franka/assets/franka_reach_v0.xml',
        'config_path': '../envs/arms/franka/assets/franka_reach_v0.config',
        'robot_site_name': "end_effector",
        'target_site_name': "target",
        'target_xyz_range': {'high':[0.3, .5, 1.2], 
                             'low':[-.3, .1, .8]
                            }
    }
)

Lets inspect one of the passed configurations

In [ ]:
env = gym.make('DemoFrankaReachRandom-v0')
print("Target randomization range::\n", env.target_xyz_range)

### 2. Registering env variants
(note: there is a seperate tutorial specifically on how to customize obs/proprio/extero of an env)

While every RoboHive env is packaged with care, research projects often require flexibility to customize the prepackaged environments. RoboHive provides functionality to easily create variants of pre-registered environments. We have found this functionality to be really useful when multiple closely related envs are required. For example -- env variations during hyper parameter sweeps, testing a policy on different env conditions, system identification.

In [ ]:
from robohive.envs.env_variants import register_env_variant
base_env_name = "FK1_RelaxFixed-v4"

# Register a variant of the kitchen env
base_env_variants={
    'max_episode_steps':50,                     # special key
    'obj_goal': {"lightswitch_joint": -0.7},    # obj_goal keys will be updated
}
variant_env_name = register_env_variant(env_id=base_env_name, variants=base_env_variants)

Lets inspect the two env variants. Pay attention to two details -- name and the updated specs.
1. Note that env-variant picks up a new unique name to distinguish itself from the original env. Unique names can also be explicitly provided using `variant_id`
2. Variant env picks up the missing details from the base env. Information is merged by defaults. `override_keys` can be used to fully override any keys

In [ ]:
print(f"Base-env kwargs: {base_env_name}")
print(gym.envs.registry.env_specs[base_env_name]._kwargs)
print(f"\nEnv-variant kwargs: {variant_env_name}")
print(gym.envs.registry.env_specs[variant_env_name]._kwargs)

### 3. Passing `kwargs` during make (not advised)
In the examples below `kwargs` are used to change/boost the prespecified randomization range of the target

In [ ]:
env_kwargs = {'target_xyz_range' : 
                 {'high':[0.4, .6, 1.3], 
                  'low':[-.2, .0, .7]
                 }
             }
env_new_range = gym.make('DemoFrankaReachRandom-v0', **(env_kwargs))

Lets inspect the passed configurations between the two envs

In [ ]:
print(f"Original env: {env.id}\n target randomization range::\n", env.target_xyz_range)
print(f"Updated env: {env.id}\n target randomization range::\n", env_new_range.target_xyz_range)

In the output you will note that the two envs have different settings but have the same id. This is confusing/misleading.

**NOTE:** Passing kwargs during make is not advisable as the updated env has the same id as the original env. This leads to two potential issues 

1. *Confusion while reporting results* - Reporting results using the env's name while its configurations have been changed leads to reproducibility issues. If env's configuration changes are needed, it is recommended to instead to use env_variant to register the altered name with its own unique id. This is a very common mistake in the field. Let's fix this! 
**Recommendation**: For reporting results RoboHive recommends projects/papers to use `register_env_variant` at top of their scripts to create a unique env name `<PAPERNAME>_FrankaReachRandom-v0` if default envs are customized in any way.

2. *Confusion while usage* - two env with the same id but different properties can lead to confusion during development/usage. 